# **2252. Dynamic Pivoting of a Table**

**Table: Products**
``` sql
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| product_id  | int     |
| store       | varchar |
| price       | int     |
+-------------+---------+
```
(product_id, store) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates the price of product_id in store.

There will be at most 30 different stores in the table.

price is the price of the product at this store.
 

Important note: This problem targets those who have a good experience with SQL. If you are a beginner, we recommend that you skip it for now.

Implement the procedure PivotProducts to reorganize the Products table so that each row has the id of one product and its price in each store. The price should be null if the product is not sold in a store. The columns of the table should contain each store and they should be sorted in lexicographical order.

The procedure should return the table after reorganizing it.

Return the result table in any order.

The result format is in the following example.

 

**Example 1:**

**Input:** 
**Products table:**
``` sql
+------------+----------+-------+
| product_id | store    | price |
+------------+----------+-------+
| 1          | Shop     | 110   |
| 1          | LC_Store | 100   |
| 2          | Nozama   | 200   |
| 2          | Souq     | 190   |
| 3          | Shop     | 1000  |
| 3          | Souq     | 1900  |
+------------+----------+-------+
```
**Output:** 
``` sql
+------------+----------+--------+------+------+
| product_id | LC_Store | Nozama | Shop | Souq |
+------------+----------+--------+------+------+
| 1          | 100      | null   | 110  | null |
| 2          | null     | 200    | null | 190  |
| 3          | null     | null   | 1000 | 1900 |
+------------+----------+--------+------+------+
```
**Explanation:** 
- We have 4 stores: Shop, LC_Store, Nozama, and Souq. We first order them lexicographically to be: LC_Store, Nozama, Shop, and Souq.

- Now, for product 1, the price in LC_Store is 100 and in Shop is 110. For the other two stores, the product is not sold so we set the price as null.

- Similarly, product 2 has a price of 200 in Nozama and 190 in Souq. It is not sold in the other two stores.

- For product 3, the price is 1000 in Shop and 1900 in Souq. It is not sold in the other two stores.

**Solution-1:**

``` sql
CREATE PROCEDURE PivotProducts()
BEGIN
	# Write your MySQL query statement below.
    set session group_concat_max_len = 1000000;
    
    set @case_stmt = null;
    
    select group_concat(distinct concat('sum(case when store = "',store,'"then price end) as ', store))
    into @case_stmt
    FROM products;

    set @sql_query = concat('select product_id, ',@case_stmt, ' from products group by product_id');

    PREPARE final_query from @sql_query;
    execute final_query;
    deallocate prepare final_query; 
END

## **Explanation:**

``` sql
SELECT product_id,
    SUM(CASE WHEN store='store1' THEN price END) AS store1 # dynamically generated by case_stmt, where store1 is a record in column "store"
    SUM(CASE WHEN store='store2' THEN price END) AS store2,
    SUM(CASE WHEN store='store3' THEN price END) AS store3, ......
    ......
    SUM(CASE WHEN store='storen' THEN price END) AS storen
    FROM Products
    GROUP BY product_id;

--GROUP_CONCAT: concatenate all the "SUM(CASE WHEN......)" statements for the corresponding product_id
--CONCAT: used to concatenate the string, since we want the column name to be the "store" record name (eg. CONCAT("AS ", store))

``` sql
SELECT
  product_id,
  SUM(IF(store = 'store1', price, NULL)) AS store1,
  SUM(IF(store = 'store2', price, NULL)) AS store2,
  SUM(IF(store = 'store3', price, NULL)) AS store3,
  SUM(IF(store = 'store4', price, NULL)) AS store4,
  SUM(IF(store = 'store5', price, NULL)) AS store5
FROM
  Products p
GROUP BY
  product_id;